In [ ]:
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')

# Actividad de Clasificación de Imágenes mediante CNNs

En este trabajo se aborda la tarea de clasificación de imágenes utilizando diferentes arquitecturas de Redes Neuronales Convolucionales (CNNs). El objetivo principal es comparar el rendimiento de clasificación obtenido con distintas variantes del modelo.

**Metodología**
1. Conjunto de datos:
  - Se utilizará el conjunto CIFAR10, disponible en la biblioteca PyTorch (torchvision.datasets).
  - Se seleccionarán tres clases específicas del conjunto de datos.

2. Variantes del modelo;
Se implementarán las siguientes versiones de CNN:
  - Baseline: 24 canales despúes de la primera capa convolucional.
  - Width: 40 características después de la primera capa lineal.
  - Regularización: L2 (λ = 0.002).
  - Dropout: (probabilidad = 0.6)
  - Batch Normalization: 40 características después de la primera capa lineal.
  - Depth: 36 canales después de la primera capa convolucional.
  - Residual: 36 canales después de la primera capa convolucional.
  - Residual Deep: 22 canales después de la primera capa convolucional con 60 bloques.

3. Entrenamiento y evaluación
  - Se medirá el rendimiento de los clasificadores entrenados.
  - Se generarán gráficos comparativos del desempeño en los conjuntos de entrenamiento y validación.

4. Tareas adicionales
  - Comparación tiempo de entrenamiento para CPU y GPU.
  - Evaluación de Robustez ante Ruido Gaussiano.

5. Conclusiones
  - Se extraerán unas conclusiones del trabajo realizado.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import collections
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

## Introducción

Una **Red Neuronal Convolucional (CNN)**, es un tipo de algoritmo de **aprendizaje profundo** diseñado principalmente para tareas relacionadas con el reconocimiento de patrones y objetos, como la clasificación, la detección y la segmentación de imágenes.

La arquitectura de una CNN se compone de dos partes principales: una **sección convolucional** y una **sección de clasificación**.

- La parte convolucional se encarga de extraer las características propias de cada imagen, coprimiendo y reduciendo progresivamente su tamaño mediante capas convolucionales y capas de pooling.
- La parte de clasificación consta de capas totalmente conectadas donde se combinan las caracteristicas extraídas por la red para asignar una clase final a la imagen.

Las CNN están compuestas por varios tipos de capas:

- **Capas Convolucionales**: Extraen características locales (features) aplicando filtros entrenables.
- **Funciones de Activación**
- **Capas de Pooling**: Reducen la dimensión espacial, mejorando la eficiencia de la red.
- **Capas Completamente Conectadas**: Transforman las características extraídas en una predicción final.

A medida que los datos avanzan por las capas convolucionales, los mapas de características representan descripciones progresivamente más abstractas y de mayor nivel. En la fase final, las capas totalmente conectadas se encargan de clasificar la imagen completa.


## Carga y procesamiento del conjunto de datos

El conjunto de datos utilizado es **CIFAR-10**, un benchmark estándar que contiene 60.000 imágenes a color (32x32), distribuidas en 10 clases distintas.

Para este trabajo, se han seleccionado específicamente las clases: automóvil, gato y rana.

En primer lugar, las imágenes son transformadas utilizando las funciones ToTensor() y Normalize():

- ToTensor() convierte las imágenes a tensores de PyTorch.
- Normalize() estandariza los valores de los píxeles utilizando la media y la desviación estándar precalculadas de CIFAR-10. Esta normalizacion se aplica por canal (RGB).

A continuación, se filtran las clases para obtener un subconjunto de datos con las tres clases seleccionadas.

Finalmente se crean los dataloaders.




In [ ]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

DATA_PATH = '../data-unversioned/p1ch6/'
SELECTED_CLASSES = [1, 3, 6]  # automobile, cat, frog
CLASS_NAMES = ['automobile', 'cat', 'frog']
LABEL_MAP = {1: 0, 3: 1, 6: 2}

# Valores de normalización (media y std calculados para CIFAR-10)
NORMALIZE_MEAN = (0.4915, 0.4823, 0.4468)
NORMALIZE_STD = (0.2470, 0.2435, 0.2616)

#Transformaciones
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(NORMALIZE_MEAN, NORMALIZE_STD)
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(NORMALIZE_MEAN, NORMALIZE_STD)
])

#Carga de dataset
cifar10_train = datasets.CIFAR10(
    DATA_PATH,
    train=True,
    download=True,
    transform=train_transform
)

cifar10_val = datasets.CIFAR10(
    DATA_PATH,
    train=False,
    download=True,
    transform=val_transform
)


#Filtrado

def filter_classes(dataset, selected_classes, label_map):
    """Filtra dataset manteniendo solo las clases seleccionadas"""
    data = [(img, label_map[label])
            for img, label in dataset
            if label in selected_classes]
    return data

cifar3_train = filter_classes(cifar10_train, SELECTED_CLASSES, LABEL_MAP)
cifar3_val = filter_classes(cifar10_val, SELECTED_CLASSES, LABEL_MAP)


train_loader = DataLoader(cifar3_train, batch_size=32, shuffle=True)
val_loader = DataLoader(cifar3_val, batch_size=32, shuffle=False)

#Verificación de datos
sample_img, sample_label = cifar3_train[0]
print(f"\nInformación del dataset:")
print(f"- Número de muestras entrenamiento: {len(cifar3_train)}")
print(f"- Número de muestras validación: {len(cifar3_val)}")
print(f"- Shape de las imágenes: {sample_img.shape}")


## 3. Definición de modelos

### Modelo Baseline

El modelo **Baseline** implementa una Red Neuronal Convolucional secuencial, utilizando filtros bidimensionales de tamaño 3x3.  Su arquitectura combina capas convolucionales, de activación, pooling y una capa completamente conectada para clasificación.

Como functión de activación se utiliza ReLU (Función lineal rectificada). Esta función introduce no linealidad, permitiendo el aprendizaje de patrones complejos en grandes volúmenes de datos. Mitiga el problema de la desaparición de gradientes (a diferencia de las funciones sigmoide y tanh).

**Arquitectura detallada**

- Capa convulacional Conv2d(3, 24, kernel_size=3, padding=1):
  - Entrada: imagen con tres canales (RGB), tamaño 32x32.
  - Parámetros:
    - 24 filtros de 3x3.
    - Padding = 1: Mantiene el tamaño de la imagen.
  - Salida:  Tensor de [24, 32, 32].

- ReLu() + MaxPool2d(2):
  - ReLU(): Activa solo los valores positivos e introduce no linealidad.
  - MaxPool2d(2): Reduce dimensiones a la mitad → [24, 16, 16].

- Capa convulucional Conv2d(24, 48, kenerl_size=3, padding=1):
  - Entrada: 24 canales, tamaño 16x16-
  - Parámetros:
    - 48 filtos de 3x3.
    - Padding = 1.
  - Salida: Tensor de [48, 16, 16].

- ReLU() + MaxPool2d(2):
  - Similar al paso anterior. Salida: [48, 8, 8].

- Flatten():
  - Convierte el tensor 3D en un vector 1D.
  - Entrada: [batch_size, 48, 8, 8]
  - Salida: [batch_size, 3072] (48x8x8)

- Capa completamente conectada:
  - Linear(3072, 128): Conecta las 3072 entradas a 128 neuronas.
  - ReLU: Introduce no linealidad
  - Linear(128, 3): Capa final para clasificación en 3 clases (coche, gato, rana).

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3, 24, kernel_size=3, padding=1),  # 1ª conv
    nn.ReLU(),
    nn.MaxPool2d(2),                             # Reduce a la mitad --> [24, 16, 16]

    nn.Conv2d(24, 48, kernel_size=3, padding=1), # 2ª conv
    nn.ReLU(),
    nn.MaxPool2d(2),                             # Reduce a la mitad --> [48, 8, 8]

    nn.Flatten(),
    nn.Linear(48*8*8, 128),                      # capa oculta
    nn.ReLU(),
    nn.Linear(128, 3)                            # salida para 3 clases
)

numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

In [ ]:
sample_img, _ = cifar3_train[0]  # Una imagen del conjunto filtrado
sample_img = sample_img.unsqueeze(0)  # Añade dimensión batch: [1, 3, 32, 32]

# Imagen original
plt.imshow(np.transpose(sample_img.squeeze().numpy(), (1, 2, 0)))
plt.title("Imagen original (entrada)")
plt.axis('off')
plt.show()

# Pasa la imagen por las primeras capas (hasta el segundo MaxPool)
with torch.no_grad():
    conv1 = model[0](sample_img)   # Conv2d(3→24)
    relu1 = model[1](conv1)
    pool1 = model[2](relu1)

    conv2 = model[3](pool1)        # Conv2d(24→48)
    relu2 = model[4](conv2)
    pool2 = model[5](relu2)

# Mapas de activación tras la segunda capa convolucional
def show_feature_maps(tensor, title):
    fig, axs = plt.subplots(3, 6, figsize=(12, 6))
    for i in range(18):
        axs[i // 6, i % 6].imshow(tensor[0, i].numpy(), cmap='gray')
        axs[i // 6, i % 6].axis('off')
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

show_feature_maps(conv1, "Salida Conv1 (24 canales)")
show_feature_maps(pool1, "Tras MaxPool1")
show_feature_maps(conv2, "Salida Conv2 (48 canales)")
show_feature_maps(pool2, "Tras MaxPool2")


### Modelo width
El modelo **Width** tiene 40 neuronas despúes de la primera capa lineal,  en lugar de las 128 neuronas que se usan en el modelo base.

Esto reduce la dimensión del espacio latente aprendido antes de la salida.

In [ ]:
model_width = nn.Sequential(
    nn.Conv2d(3, 24, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Conv2d(24, 48, kernel_size=3, padding=1),
    nn.BatchNorm2d(48),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Flatten(),
    nn.Linear(48 * 8 * 8, 40),       # ← 40 features
    nn.ReLU(),
    nn.Linear(40, 3)
)


### Modelo Batch Normalization

El modelo **Batch Normalization** añade normalización por lotes después de cada capa convolucional.

In [ ]:
model_bn = nn.Sequential(
    nn.Conv2d(3, 24, kernel_size=3, padding=1),
    nn.BatchNorm2d(24),              # ← se normaliza cada canal
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Conv2d(24, 48, kernel_size=3, padding=1),
    nn.BatchNorm2d(48),              # ← también en la segunda conv
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Flatten(),
    nn.Linear(48*8*8, 40),
    nn.ReLU(),
    nn.Linear(40, 3)
)

### Modelo con Dropout

El modelo **Dropout** incorpora la técnica de regularización dropout para mitigar el sobreajuste (overfitting) durante el entrenamiento. Dropout desactiva aleatoriamente un porcentaje de neuronas durante el entrenamiento (en este caso el 60%) en la capa designada durante cada iteración de entrenamiento. Esto reduce la dependencia del modelo hacia neuronas específicas, promoviendo así una mejor generalización.

La arquitectura es similar al modelo baseline, unicamente añadiendo una capa Dropout(p=0.6) después de la primera capa lineal.

In [ ]:
model_dropout = nn.Sequential(
    nn.Conv2d(3, 24, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Conv2d(24, 48, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Flatten(),
    nn.Linear(48 * 8 * 8, 128),
    nn.ReLU(),
    nn.Dropout(p=0.6),          # ← Dropout
    nn.Linear(128, 3)
)


### Modelo con mayor profondidad y residual

En el modelo **Depth** se aumenta la profundidad de la red convolucional, es decir, se añaden más capas convolucionales permitiendo que la red aprenda características más complejas de los datos.

Los cambios de esta versión incluyen, obtener 36 canales despúes de su primera capa convolucional y añadir una capa convulacional adicional con un mayor número de filtros.

In [ ]:
model_depth = nn.Sequential(
    nn.Conv2d(3, 36, kernel_size=3, padding=1),   # ← 36 canales
    nn.ReLU(),
    nn.MaxPool2d(2),                              # 32 → 16

    nn.Conv2d(36, 72, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),                              # 16 → 8

    nn.Conv2d(72, 128, kernel_size=3, padding=1),  # capa extra
    nn.ReLU(),
    nn.MaxPool2d(2),                              # 8 → 4

    nn.Flatten(),
    nn.Linear(128 * 4 * 4, 128),
    nn.ReLU(),
    nn.Linear(128, 3)
)


### Modelo residual

El modelo **Residual** implementa bloques residuales. Los bloques residuales fueron introducidos en ResNet, permitiendo conexiones de saltos (skip connections) que permiten el flujo directo del gradiente, mitigando el problema del *vanishing gradient* en redes profundas.

En redes tradicionales, cuando el gradiente se retropropaga, puede volverse extremadamente pequeño debido a la repetida multiplicación en capas sucesivas. Las conexiones residuales simplifican este proceso al sumar la entrada original al resultado de las convoluciones, usando menos capas durante la etapa de entrenamiento inicial. Esto permite que la red aprenda ajustes (residuos) en lugar de construir toda la información desde cero.

En el bloque residual, se aplica dos capas convolucionales (cv1 y cv2) al mismo tensor de entrada. Luego, la salida de la segunda convolución se suma a la entrada original, creando la conexión residual.

En este modelo se utilizan dos bloques residuales con 36 canales cada uno. Cada bloque realiza dos convoluciones seguidas de la conexión residual.

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)

    def forward(self, x):
        return self.relu(self.conv2(self.relu(self.conv1(x))) + x)

class ResidualCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.start = nn.Sequential(
            nn.Conv2d(3, 36, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.res1 = ResidualBlock(36)
        self.res2 = ResidualBlock(36)
        self.end = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(36*8*8, 128),
            nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, x):
        x = self.start(x)
        x = self.res1(x)
        x = self.res2(x)
        return self.end(x)

### Modelo residual profundo

El modelo **Residual Deep** tiene 22 canales despues de su primera capa convolucional y usa 60 bloques residuales consecutivos.

In [ ]:
class ResidualDeepCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.start = nn.Sequential(
            nn.Conv2d(3, 22, kernel_size=3, padding=1), #22 Canales después de su primera capa convolucional
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.res_blocks = nn.Sequential(*[ResidualBlock(22) for _ in range(60)]) #60 Bloques residuales
        self.end = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(22*8*8, 128),
            nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, x):
        x = self.start(x)
        x = self.res_blocks(x)
        return self.end(x)

## Entrenamiento del modelo

En este apartado se implementa el proceso completo de entrenamiento de la red neuronal. Cada modelo se entrena durante 10 ciclos completos de entrenamiento. La función train_model se encarga de:

- **Configuración inicial**: Detecta automáticamente si hay GPU disponible (CUDA). En caso contrario utiliza CPU.
- **Función de pérdida - CrossEntropyLoss()**: Evalúa el rendimiento comparando las predicciones resultantes con las etiquetas reales.
- **Optimizador SGD**: Actualiza los pesos del modelo en función del gradiente calculado. Aplica regularización L2.
- **Ciclo de entrenamiento**: Por cada época (iteración completa sobre los datos):
  - Fase de entrenamiento: Procesa los datos por lotes (batches) y calcula predicciónes y pérdidas.
  - Fase de validación: Evalúa el modelo y calcula métricas de precisión.

In [ ]:
def train_model(model, train_loader, val_loader, num_epochs=10, lr=0.001, l2_lambda=0.0):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=l2_lambda)

    train_acc_list = []
    val_acc_list = []

    for epoch in range(num_epochs):
        model.train()
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()           #Limpia los gradientes
            outputs = model(images)         #Forward: Predicción
            loss = loss_fn(outputs, labels) #Calcula pérdida
            loss.backward()                 #Calcula gradientes
            optimizer.step()                #Actualiza pesos

            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        train_acc_list.append(train_acc)

        # Evaluación
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                preds = outputs.argmax(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_acc = correct / total
        val_acc_list.append(val_acc)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Acc: {train_acc:.4f} - Val Acc: {val_acc:.4f}")

    return train_acc_list, val_acc_list


In [ ]:
results = {}

# Modelo baseline
print("Baseline:")
results["Baseline"] = train_model(model, train_loader, val_loader,)

# Modelo width
print("Width:")
results["Width"] = train_model(model_width, train_loader, val_loader)

#Modelo Batch Normalization
print("BN:")
results["BN"] = train_model(model_bn, train_loader, val_loader)

# Modelo Dropout
print("Dropout:")
results["Dropout"] = train_model(model_dropout, train_loader, val_loader)

# Modelo Depth
print("Depth:")
results["Depth"] = train_model(model_depth, train_loader, val_loader)

# Modelo Residual
print("Residual:")
residual_model = ResidualCNN()
results["Residual"] = train_model(residual_model, train_loader, val_loader)

# Modelo Residual Deep
print("Residual Deep:")
residual_deep_model = ResidualDeepCNN()
results["Residual_Deep"] = train_model(residual_deep_model, train_loader, val_loader, num_epochs=6)

# Modelo con L2 regularization (mismo que baseline pero con lambda)
print("L2 Regularization:")
model_l2 = nn.Sequential(
    nn.Conv2d(3, 24, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(24, 48, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(48 * 8 * 8, 128),
    nn.ReLU(),
    nn.Linear(128, 3)
)
results["L2 Regularization"] = train_model(model_l2, train_loader, val_loader, l2_lambda=0.002)


In [ ]:
plt.figure(figsize=(12, 6))

for name, (train_acc, val_acc) in results.items():
    plt.plot(train_acc, linestyle='--', label=f"{name} (Train)")

for name, (train_acc, val_acc) in results.items():
    plt.plot(val_acc, linestyle='-', label=f"{name} (Val)")

plt.title("Precisión de Entrenamiento y Validación por Modelo")
plt.xlabel("Época")
plt.ylabel("Precisión")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

A continuación, se presenta un análisis comparativo del rendimiento de los modelos:

**Modelos con mejor desempeño: Width y Batch Normalization**

- Width: Precisión de validación 0.87, con una curva de aprendizaje estable y sin señales de sobreajuste. Este modelo supera al baseline desde las primeras iteraciones gracias a un espacio latente más compacto (40 neuronas en lugar de 128), lo que mejora su capacidad de generalización.
- Batch Normalization: Muestra el mejor desempeño, alcanzando una validación final de 0.88. Estabiliza el entrenamiento al normalizar las activaciones.

**Modelo Baseline (Referencia)**
- El modelo Baseline alcanza 0.79 en la última iteración. Su rendimiento es consistente pero inferior al de Width y BN.

**Regularización (Dropout y L2)**

- Dropout: Mejora de 0.53 a 0.76. Muestra un aprendizaje lento, ya que desactiva un 60% de las neuronas durante el entrenamiento, lo que limita su capacidad máxima, aunque mejora la generalización.
- L2 Regularization: Muestra una mejora constante (0.53 → 0.77). Penaliza pesos grandes, evitando sobreajuste sin sacrificar tanto rendimiento como Dropout.

**Modelos Profundos: Depth y Residual**

- Depth: Comienza con 0.33 de validación y llega a 0.71, mostrando underfitting al inicio y un aprendizaje más lento.
- Residual: Llega hasta 0.83 en validación, mostrando que las conexiones de salto mejoran el flujo de gradiente y aceleran el aprendizaje en comparación con Depth.

**Residual Deep**

- La precisión final después de 6 epocas es 0.80. Esto muestra un buen rendimiento en menos épocas, pero con ligera inestabilidad al final.

**Conclusiones finales**

En conclusión, los modelos Width y Batch Normalization muestran los mejores rendimientos, con BN mostrando un equilibrio ligeramente mejor entre ajuste y robustez.

Estos resultados pueden ser debidos a:
1. El tamaño reducido del dataset (solo 3 clases de CIFAR-10): Limitó la efectividad de modelos complejos (Depth, Residual Deep), que requieren más datos para evitar sobreajuste.
2. Capacidad del modelo y regularización: Arquitecturas más anchas (Width) o con normalización (BN) lograron un mejor equilibrio entre aprendizaje y generalización.
3. Sencillez vs. complejidad: Modelos como Width y BN priorizaron la extracción de características esenciales sin memorizar ruido, a diferencia de alternativas más complejas que tendieron a sobreajustarse.

En resumen, la simplicidad controlada (Width) y la normalización (BN) son estrategias ganadoras cuando los datos son limitados.



## Tareas adicionales

### Comparación tiempo de entrenamiento para CPU y GPU

En esta sección se ha realizado una comparación entre el tiempo de entramiento en CPU y en GPU, para los modelos baseline, width y bn.

In [ ]:
def train_with_timing(model, train_loader, val_loader, device_str='cpu', epochs=5, lr=0.001, l2_lambda=0.0):
    device = torch.device(device_str)
    model = model.to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=l2_lambda)

    start_time = time.time()

    for epoch in range(epochs):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                _ = model(images)

    total_time = time.time() - start_time
    return total_time


In [ ]:
times = {}

models = {
    "Baseline": model,
    "Width": model_width,
    "BN": model_bn
}

# Comparación CPU vs GPU
for name, mdl in models.items():
    print(name)
    cpu_time = train_with_timing(mdl, train_loader, val_loader, device_str='cpu')
    print(f"CPU time: {cpu_time}")

    mdl.apply(lambda m: m.reset_parameters() if hasattr(m, 'reset_parameters') else None)

    gpu_time = train_with_timing(mdl, train_loader, val_loader, device_str='cuda')
    print(f"GPU time: {gpu_time}")

    times[name] = {"CPU": cpu_time, "GPU": gpu_time}

Baseline
CPU time: 75.59359097480774
GPU time: 6.359274864196777
Width
CPU time: 70.76189064979553
GPU time: 6.162753105163574
BN


In [ ]:
labels = list(times.keys())
cpu_times = [times[m]["CPU"] for m in labels]
gpu_times = [times[m]["GPU"] for m in labels]

x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots(figsize=(10,6))
bars1 = ax.bar(x - width/2, cpu_times, width, label='CPU')
bars2 = ax.bar(x + width/2, gpu_times, width, label='GPU')

ax.set_ylabel('Tiempo (segundos)')
ax.set_title('Comparación de tiempos de entrenamiento (CPU vs GPU)')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45)
ax.legend()
ax.grid(True)

plt.tight_layout()
plt.show()


Como se puede observar en la gráfica anterior, el tiempo de entrenamiento en GPU es significativamente menor que el tiempo en CPU.
- **Entrenamiento CPU**: Los tiempos se sitúan entre 77 y 83 segundos por modelo. El proceso en CPU es mucho más lento, debido a la naturaleza secuencial y limitada de las operaciones en CPU.
- **Entrenamiento GPU**: Los tiempos se reducen drásticamente a 6-7 segundos, lo que supone una aceleración de más de 10 veces en comparación con la CPU.

### Evaluación de robustez ante Ruido Gaussiano

Con el objetivo de evaluar la capacidad de los modelos frente al ruido, se ha evaluado el rendimiento en el conjunto de validación añadiendo **ruido gaussiano** a las imágenes.

El ruido se genera con **torch.randn**, que produce valores con distribución normal (media 0, desviación estándar 1), y se escala por un factor σ.

In [ ]:
def evaluate_with_noise(model, val_loader, noise_levels, device="cuda"):
    model.eval()
    model.to(device)
    accuracies = []

    with torch.no_grad():
        for sigma in noise_levels:
            correct = 0
            total = 0
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                # Añadir ruido gaussiano
                noisy_images = images + sigma * torch.randn_like(images)
                noisy_images = torch.clamp(noisy_images, 0, 1)

                outputs = model(noisy_images)
                preds = outputs.argmax(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

            acc = correct / total
            accuracies.append(acc)
            print(f"Ruido sigma={sigma:.2f} -> Precisión: {acc:.4f}")

    return accuracies



noise_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]

models = {
    "Width": model_width,
    "Dropout": model_dropout,
    "Residual": residual_model
}

results_noise={}

for name, mdl in models.items():
  print(f"Modelo {name}")
  acc_noise = evaluate_with_noise(mdl, val_loader, noise_levels)
  results_noise[name] = acc_noise

plt.figure(figsize=(8, 5))
for name, acc_noise in results_noise.items():
    plt.plot(noise_levels, acc_noise, marker='o', label=name)

plt.title("Comparación de Robustez ante Ruido Gaussiano")
plt.xlabel("Desviación estándar del ruido (σ)")
plt.ylabel("Precisión en Validación")
plt.grid(True)
plt.legend()
plt.show()


Los resultados muestran como la precisión de los modelos Width, Dropout y Residual se ve afectada al introducir ruido gaussiano en el conjunto de validación.

- **Modelo Width**: La precisión empieza en 0.46 con datos sin ruido, y alcanza 0.56 en 0.3, para luego descender. Este comportamiento sugiere que pequeñas cantidades de ruido pueden actuar como regularización, mejorando ligeramente la capacidad del modelo para generalizar. Si el ruido es ecesivo, la precisión disminuye.
- **Modelo Dropout**: La precisión es muy estable en todo el rango de ruido (0.47-0.48), mostrando casi ninguna variación. Esto ocurre porque Dropout ya entrena con "ruido interno" (desactivación aleatoria de neuronas), por lo que el modelo es más robusto a perturbaciones externas.
- **Modelo Residual**: La precisión comienza en 0.45 y desciende progresivamente hasta 0.39 al aumentar el ruido. El modelo Residual parece más sensible al ruido, probablemente porque las conexiones residuales amplifican ciertas características de la imagen que se ven degradadas cuando se introducen perturbaciones.

## Conclusión

Tras definir y entrenar distintas arquitecturas de CNN utilizando el conjunto de datos CIFAR-10 (centrándonos en tres clases específicas), hemos obtenido las siguientes conclusiones:

- Los modelos más efectivos son Batch Normalization y Width. BN alcanzó la mejor precisión en validación (~0.88), beneficiándose de la normalización de activaciones que estabiliza y acelera el entrenamiento.
- Dropout (p=0.6) y L2 Regularization ayudaron a reducir el sobreajuste, aunque con una ligera pérdida en la precisión máxima.
- Modelos profundos (Depth y Residual Deep) mostraron un aprendizaje más lento y una mayor necesidad de datos o de más épocas para alcanzar su máximo potencial.
- Residual, con conexiones de salto, ofreció una mejora notable respecto a Depth (0.83 vs. 0.71 en validación), confirmando la importancia de las conexiones residuales para el flujo de gradiente.

En el análisis de robustez con ruido gaussiano, Dropout resultó ser el modelo más estable (manteniendo ~0.47 de precisión en todo el rango de ruido), mientras que Width mostró un interesante pico de rendimiento con ruido moderado, lo que indica una buena capacidad de generalización.

El trabajo confirma que modelos más simples pero bien regularizados (Width o BN) pueden superar en rendimiento y estabilidad a arquitecturas más profundas, especialmente cuando el conjunto de datos es reducido.

In [ ]:
!jupyter nbconvert --to PDF "/content/drive/MyDrive/Colab Notebooks/ObjectClassificationActivity.ipynb"